In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import preprocessing as p
import math
from scipy.stats import norm

In [2]:
df1 = pd.read_csv("sample/2014_01_01.csv")
df2 = pd.read_csv("sample/2014_01_02.csv")
df3 = pd.read_csv("sample/2014_01_03.csv")

In [3]:
p.to_sec(df1)
p.to_sec(df2)
p.to_sec(df3)
p.to_ID(df1)
p.to_ID(df2)
p.to_ID(df3)

## Similarity Check
### ID-agnostic mode

In [5]:
l = []
l += df1[df1.ID == 1]['sec'].tolist()
l += df2[df2.ID == 1]['sec'].tolist()
l += df3[df3.ID == 1]['sec'].tolist()

In [6]:
l.sort()

In [9]:
DIM = 3

#learned by rl
#가장 많이 차이나는 컬럼(?)이 몇초만큼 차이나는가
SIGMA = 5000

In [10]:
n_tuples = []
for i in range(len(l)-DIM):
    n_tuples.append(np.array(l[i:i+DIM]))

In [11]:
Tr = n_tuples[-1]
#편의상 하루 중 가장 마지막 기록으로 설정함.
#추후에 실시간으로 돌릴때에는 최근값으로 다시 설정해야함

In [13]:
predicted = []
for i,val in enumerate(n_tuples[:-1]):
    #maximum norm
    if np.linalg.norm(Tr-val, ord = np.inf) < SIGMA:
        predicted.append(n_tuples[i+1][0])

In [14]:
predicted

[81021, 81021, 81259, 82808, 82811, 83035]

일별 초단위로 계산: 특정 시간대에 대한 특징 반영 가능. 대신 다음날로 넘어가는 부분의 연속성은 파악하기 어렵다

### ID-aware mode
To be continued..

## Predictions Phase

In [15]:
DELTA = 900  #second
EPS_prd = 0.3 #ratio

In [16]:
R_prd = []
tmp = []
for i,val in enumerate(predicted[:-1]):
    tmp.append(val)
    if(predicted[i+1] - val > DELTA):
        #separate
        if len(tmp)/len(predicted) > EPS_prd:
            R_prd.append(np.average(tmp))
        tmp = []
tmp.append(predicted[-1])
if len(tmp)/len(predicted) > EPS_prd:
    R_prd.append(np.average(tmp))

In [17]:
R_prd

[81100.33333333333, 82884.66666666667]

## Duty Cycle adaptation

In [ ]:
#persentage
DC_MIN = 2
DC_MAX = 20
DC_DEF = 8

#seconds
L_PRD = 900
L_EXP = 360

#standart deviation
SIGMA_PRD = L_PRD/math.sqrt(8*math.log(DC_MAX/DC_DEF))
SIGMA_EXP = L_EXP/math.sqrt(8*math.log(DC_DEF/DC_MIN))
A_PRD = DC_MAX*SIGMA_PRD*math.sqrt(2*math.pi)
A_EXP = DC_DEF*SIGMA_EXP*math.sqrt(2*math.pi)

#learned by rl
GAMMA = 3

In [ ]:
def Seren(t, t_s, t_e):
    n_sub = math.ceil((t_e-t_s)/GAMMA)
    if n_sub <= 1:
        return 0
    R_exp = []
    t_exp = 99999 #some big number
    for i in range(1,n_sub):
        t_tmp = t_s+i*(t_e-t_s)/n_sub
#         R_exp.append(t_tmp)
        if abs(t-t_exp) > (t-t_tmp):
            t_exp = t_tmp
    return A_EXP*norm(t_exp, SIGMA_EXP).pdf(t)

In [ ]:
#Adaptive Duty Cycling
def Adaptive_DC(t,t_cur,t_prev,R):
    #t: 듀티사이클을 계산하고 있는 시점(추정)
    #t_cur: current estimated contact time from R_prd
    #t_prev: previous ~~
    if R_prd == []:
        return DC_DEF
    C_prd = A_PRD*norm(t_cur, SIGMA_PRD).pdf(t)
    C_exp = Seren(t, t_prev, t_cur)
    C = C_prd+C_exp
    if C<DC_MIN:
        C = DC_MIN
    return C
    

,timestamp,client,AP,sec,ID
0,2014-01-01 00:00:24,b7f22f3abd9f367af65bd70b4b328e0c35f7cfac,Bldg11AP21,24,0
1,2014-01-01 00:00:30,336501aeaedff0462f3cd55089fa433dbbfff493,Bldg44AP3,30,1
2,2014-01-01 00:00:35,4b912f490ddc1f289f35def68a1662bcc5089a1a,Bldg48AP65,35,2
3,2014-01-01 00:00:40,8e10f9d5aa9c229baae2f745abd25b9c9522ed64,Bldg25AP19,40,3
4,2014-01-01 00:00:51,680fc4c2bd6b252fd6e1a67ad7aa69cdbd3d6e0f,Bldg25AP5,51,4
...,...,...,...,...,...
12724,2014-01-01 23:58:36,34a2fe9462c797cfc9916b7aa84bbe4a7b42327a,Bldg44AP15,86316,423
12725,2014-01-01 23:59:03,ec438383187d2fb4a00d4d7cd235494729d05198,Bldg44AP4,86343,42
12726,2014-01-01 23:59:14,2a9553acf404141b23ce2361d2d58e0ae7b72bb4,Bldg44AP64,86354,33
12727,2014-01-01 23:59:35,ca2a3933771ce90b5368dfdb95d1959f66fe0150,Bldg14AP12,86375,85
